In [9]:
import re
import html
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

In [2]:
train_df = pd.read_csv('../data/train.csv')
train_df.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,Office Manager,"GR, I, Athens",Administration,NaN,Workable is a venture-backed startup making cl...,We are looking for an experienced office manag...,"Excellent use of English, speaking and writing...",Our goal is to create a company where employee...,0,1,1,NaN,NaN,NaN,NaN,NaN,0
1,Front End Engineer,"US, CA, Emeryville",Engineering,NaN,Abakus is a software platform built on game th...,As Front End Engineer you will be leading the ...,3-5 years of UI/UX development experience with...,Competitive salaryStock optionsComprehensive b...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Engineering,0
2,Software Engineer,"US, MA, Wilmington",NaN,0-130000,NaN,Our client is one of the leading SaaS-based pr...,At least 5 years experience developing large-s...,NaN,0,0,0,Full-time,NaN,Bachelor's Degree,NaN,NaN,0
3,Product Manager,"US, CA, Manhattan Beach",NaN,NaN,The Boston Consulting Group (#URL_45423e1e1670...,We are looking for a Product Manager to be a p...,BASIC JOB REQUIREMENTS:Bachelors Degree form a...,The Boston Consulting Group (BCG) is a global ...,0,1,0,Full-time,Associate,Bachelor's Degree,Computer Software,Product Management,0
4,Sales Intern,"US, NY, New York",Sales,NaN,NaN,"Lean Startup Machine has trained over 25,000 a...",Experience in sales preferredInterest in tech ...,Be part of a growing and global team that has ...,0,0,1,Part-time,Internship,Some College Coursework Completed,Management Consulting,Sales,0


In [3]:
test_df = pd.read_csv('../data/test.csv')
test_df.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function
0,Optician / Sales,"US, MD, Bethesda",NaN,NaN,NaN,We are a long term well established Optometric...,The ideal person should be highly self motivat...,"We offer a competitive compensation package, w...",0,0,0,Full-time,NaN,NaN,"Health, Wellness and Fitness",Sales
1,Payroll Data Coordinator Positions - Earn $100...,"US, TX, Abbott",NaN,NaN,NaN,We are a full-service marketing and staffing f...,RequirementsAll you need is access to the Inte...,This is an entry level position and we offer f...,1,0,0,NaN,NaN,NaN,NaN,NaN
2,Dispatcher,"US, TX, AUSTIN",CALLCENTER TECH,NaN,Our globally connected world has forced busine...,We currently have an opening for a Dispatcher ...,·High School diploma/GEDSuccessfully complete ...,"POSITION TYPE: FULL TIMEHEALTH CARE, DENTAL , ...",0,1,1,Full-time,Entry level,High School or equivalent,Retail,Customer Service
3,Content Marketing Manager,"US, NY, New York",Marketing,NaN,Indicative is changing the way internet busine...,Indicative is changing the way internet busine...,Essential:Unparalleled ability to communicate ...,"Great healthcare and employee benefits, top of...",0,1,1,Full-time,Associate,Bachelor's Degree,NaN,Marketing
4,Talent Sourcing Specialist,"NZ, N, Auckland",NaN,NaN,EROAD was established to modernise New Zealand...,"EROAD is a fully integrated technology, road c...",Ideally you will have several years’ experienc...,Competitive salary and package.Flexibility - h...,0,1,0,Full-time,Associate,Unspecified,Information Technology and Services,Human Resources


In [4]:
columns = train_df.columns.tolist()
columns.remove('fraudulent')

In [10]:
columns

['title',
 'location',
 'department',
 'salary_range',
 'company_profile',
 'description',
 'requirements',
 'benefits',
 'telecommuting',
 'has_company_logo',
 'has_questions',
 'employment_type',
 'required_experience',
 'required_education',
 'industry',
 'function']

In [5]:
def solve(row):
    text = ''
    for column in columns:
        if column in ['company_profile', 'description', 'requirements', 'benefits']:
            text += (column + '' + str(row[column])[:100] + ' ')
        else:
            text += (column + '' + str(row[column]) + ' ')
    return text[:-1]

In [6]:
train_df['text'] = train_df.apply(lambda x: solve(x), axis=1)
test_df['text'] = test_df.apply(lambda x: solve(x), axis=1)

In [7]:
# texts = train_df[pd.notnull(train_df['company_profile'])].company_profile.tolist()
# max:884, 0.9:209
# texts = train_df[pd.notnull(train_df['description'])].description.tolist()
# max:2074, 0.9:324
# texts = train_df[pd.notnull(train_df['requirements'])].requirements.tolist()
# max:1612, 0.9:180
# texts = train_df[pd.notnull(train_df['benefits'])].benefits.tolist()
# max:678, 0.9:116
# 截取前，max:2621, 0.9:659
# 截取后，max:236, 0.9:84
texts = train_df.text.tolist()
max_len = 0
df_len_list = []
for i in range(len(texts)):
    text = []
    text = texts[i].split(' ')
    df_len_list.append(len(text))
    max_len = max(max_len, len(text))
print(max_len)
sorted_len = sorted(df_len_list)
sorted_len[int(len(sorted_len)*0.9)]

236


84

In [8]:
train_df.fraudulent.value_counts()

0    16914
1      766
Name: fraudulent, dtype: int64